In [ ]:
##------------plot Tremors with possible stations-------------------------##
## plot three components seismic wave, high& low frequencies and hydrophone (DPG)

import obspy
from obspy.clients.fdsn import Client
client = Client("USGS")
from obspy import UTCDateTime
import numpy as np
from obspy.geodetics import locations2degrees
from obspy.geodetics import gps2dist_azimuth
from obspy import read

import scipy
import scipy.signal as scisignal
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from obspy.taup import TauPyModel
model = TauPyModel(model="iasp91")

from PIL import Image
from IPython.display import Image
from obspy.signal.trigger import classic_sta_lta
from obspy.signal.trigger import plot_trigger

import os
# import geopy.distance
import glob
import obspy.signal.filter
import pandas as pd
import glob
import csv
import pickle
import datetime
import matplotlib.dates as mdates
import cartopy.crs as ccrs

In [ ]:
obs = pd.read_csv('Seismometer_station',sep ='   ',header=None)
trench = np.genfromtxt('trench.xyz')
sta_name = []
loc_name = []
rx = -157; ry = 55; rc = 3;
for istation in obs.index:
    disr = np.sqrt(np.square(obs.loc[istation,3]-rx)+np.square(obs.loc[istation,2]-ry))
    if disr<rc:
        sta_name.append(obs.loc[istation,1])
        loc_name.append(obs.loc[istation,0])

In [ ]:
for i in range(len(sta_name)):
    print(str(i) +':'+ sta_name[i])


In [ ]:

inum = [11,12,21,42]
sta_name2 = []
loc_name2 = []
for i in range(len(inum)):
    sta_name2.append(sta_name[inum[i]])
    loc_name2.append(loc_name[inum[i]])

In [ ]:
Direc = '2018-10-07'
datetime1 = datetime.datetime.strptime(Direc,'%Y-%m-%d')
# with open(Direc + "/tremor_day_180s_lfhf_2-8_10-16_rf4_150km_4EN.pkl","rb") as f1:
#     tremor_day = pickle.load(f1)


In [ ]:
seismic_data_H1 = []; seismic_data_H2 = []
seismic_data_HZ = []; seismic_data_HH = []
hseismic_data_H1 = []; hseismic_data_H2 = []; hseismic_data_HZ = [];
fs = []
l1 = 2; l2 = 8; h1 =10; h2 = 16;

for istation in range(len(sta_name2)):
    ##-------get data and plot data------------------##
#     sta = sta_name[istation]
#     loc = loc_name[istation]
    sta_name = glob.glob(Direc + '/' + sta_name2[istation] + '*.mseed')
    filename = sta_name[0]

    try:
        st = obspy.read(filename)
    except:
        print('No file name') 
    else:
        if len(st.select(channel='BHE'))>=1:
            st_filtH1 = st.select(channel='BHE').copy()
            st_filtH2 = st.select(channel='BHN').copy()
            st_filtHZ = st.select(channel='BHZ').copy()
        elif len(st.select(channel='HHE'))>=1:
            st_filtH1 = st.select(channel='HHE').copy()
            st_filtH2 = st.select(channel='HHN').copy()
            st_filtHZ = st.select(channel='HHZ').copy()                       
        else:
            st_filtH1 = st.select(channel='HH1').copy()
            st_filtH2 = st.select(channel='HH2').copy()
            st_filtHZ = st.select(channel='HHZ').copy() 
        
        for ist in range(len(st_filtH1)):
            st_filtH1[ist].stats.sampling_rate = round(st_filtH1[ist].stats.sampling_rate,0)             
            st_filtH2[ist].stats.sampling_rate = round(st_filtH2[ist].stats.sampling_rate,0)
        for ist in range(len(st_filtHZ)):   
            st_filtHZ[ist].stats.sampling_rate = round(st_filtHZ[ist].stats.sampling_rate,0)
        st_filtH1.merge(fill_value=0); st_hfiltH1 = st_filtH1.copy()
        st_filtH2.merge(fill_value=0); st_hfiltH2 = st_filtH2.copy()
        st_filtHZ.merge(fill_value=0); st_hfiltHZ = st_filtHZ.copy()
        print(st_filtH2)
        
        st_filtH1.filter('bandpass',freqmin=l1,freqmax=l2); seismic_data_H1.append(st_filtH1[0].data)
        st_hfiltH1.filter('bandpass',freqmin=h1,freqmax=h2); hseismic_data_H1.append(st_hfiltH1[0].data)
        
        st_filtH2.filter('bandpass',freqmin=l1,freqmax=l2); seismic_data_H2.append(st_filtH2[0].data)
        st_hfiltH2.filter('bandpass',freqmin=h1,freqmax=h2); hseismic_data_H2.append(st_filtH2[0].data)
        
        st_filtHZ.filter('bandpass',freqmin=l1,freqmax=l2); seismic_data_HZ.append(st_filtHZ[0].data)
        st_hfiltHZ.filter('bandpass',freqmin=h1,freqmax=h2); hseismic_data_HZ.append(st_filtHZ[0].data)        
        fs.append(round(st_filtH1[0].stats.sampling_rate,0))
        
        if len(st.select(channel='EDH'))>=1:
            st_filtHH = st.select(channel='EDH').copy(); st_filtHH.merge(fill_value=0)   
            st_filtHH.filter('bandpass',freqmin=l1,freqmax=l2)
            seismic_data_HH.append(st_filtHH[0].data)                        
        elif len(st.select(channel='HDH'))>=1:
            st_filtHH = st.select(channel='HDH').copy(); st_filtHH.merge(fill_value=0)   
            st_filtHH.filter('bandpass',freqmin=l1,freqmax=l2)
            seismic_data_HH.append(st_filtHH[0].data)
        else:
            st_filtHH = np.empty(len(st_filtH1[0].data))
            seismic_data_HH.append(st_filtHH)
        


In [ ]:
itime = 590
tt1 = 0
tt2 = 30
## Low frequency E component
fig,((ax1,ax5,ax2,ax4)) = plt.subplots(4,1,figsize=(15,15))
plt.rcParams['font.size'] = '12'
for ista in range(len(sta_name2)):    
    t = np.arange(itime*1.5-tt1,itime*1.5+tt2,1/60/fs[ista])   
    datetime_list = [datetime1 + datetime.timedelta(minutes = t0) for t0 in t]
    std1 = np.nanstd(seismic_data_H1[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])])
    sdata1 = seismic_data_H1[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])]/std1   
    if len(sdata1)==0:
        ax1.plot(datetime_list,np.empty(len(t))-10*ista)
    else:
        ax1.plot(datetime_list,sdata1-10*ista,c='C0')
    ax1.text(datetime_list[-1],-10*ista,sta_name2[ista])
# ax1.set_xlabel(Direc,fontsize=14)
ax1.text(datetime_list[1],7,'HH1: 2-8Hz',fontsize=14)
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax1.set_ylim(-10*(len(sta_name2)),11)
ax1.set_yticks([])
ax1.grid(True)
ax1.annotate('A',xy = (-0.03,1),xycoords=ax1.transAxes,fontsize=14)


## high frequency E component
for ista in range(len(sta_name2)):    
    t = np.arange(itime*1.5-tt1,itime*1.5+tt2,1/60/fs[ista])   
    datetime_list = [datetime1 + datetime.timedelta(minutes = t0) for t0 in t]
    std1 = np.nanstd(hseismic_data_H1[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])])
    sdata1 = hseismic_data_H1[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])]/std1   
    if len(sdata1)==0:
        ax5.plot(datetime_list,np.empty(len(t))-10*ista)
    else:
        ax5.plot(datetime_list,sdata1-10*ista,c='C0')
    ax5.text(datetime_list[-1],-10*ista,sta_name2[ista])
# ax5.set_xlabel(Direc,fontsize=14)
ax5.text(datetime_list[1],7,'HH1: 10-16Hz',fontsize=14)
ax5.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax5.set_ylim(-10*(len(sta_name2)),11)
ax5.set_yticks([])
ax5.grid(True)
ax5.annotate('B',xy = (-0.03,1),xycoords=ax5.transAxes,fontsize=14)

## N component
for ista in range(len(sta_name2)):    
    t = np.arange(itime*1.5-tt1,itime*1.5+tt2,1/60/fs[ista])   
    datetime_list = [datetime1 + datetime.timedelta(minutes = t0) for t0 in t]
    std2 = np.nanstd(seismic_data_H2[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])])
    sdata2 = seismic_data_H2[ista][int((itime*1.5-tt1)*60*fs[ista]):int((itime*1.5+tt2)*60*fs[ista])]/std2        
    if len(sdata2)==0:
        ax2.plot(datetime_list,np.empty(len(t))-10*ista)
    else:
        ax2.plot(datetime_list,sdata2-10*ista,c='C0')
    ax2.text(datetime_list[-1],-10*ista,sta_name2[ista])
# ax2.set_xlabel(Direc,fontsize=14)
ax2.text(datetime_list[1],7,'HH2: 2-8Hz',fontsize=14)
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax2.set_ylim(-10*(len(sta_name2)),11)
ax2.set_yticks([])
ax2.grid(True)
ax2.annotate('C',xy = (-0.03,1),xycoords=ax2.transAxes,fontsize=14)

## Hydrophone
for ista in range(len(sta_name2)):    
    t = np.arange(itime*1.5-tt1,itime*1.5+tt2,1/60/100)   
    datetime_list = [datetime1 + datetime.timedelta(minutes = t0) for t0 in t]
    std4 = np.nanstd(seismic_data_HH[ista][int((itime*1.5-tt1)*60*100):int((itime*1.5+tt2)*60*100)])
    sdata4 = seismic_data_HH[ista][int((itime*1.5-tt1)*60*100):int((itime*1.5+tt2)*60*100)]/std4        
    if len(sdata4)==0:
        ax4.plot(datetime_list,np.empty(len(t))-10*ista)
    else:
        ax4.plot(datetime_list,sdata4-10*ista,c='C0')
    ax4.text(datetime_list[-1],-10*ista,sta_name2[ista])
ax4.set_xlabel(Direc,fontsize=14)
ax4.text(datetime_list[1],7,'Hydrophone: 2-8Hz',fontsize=14)
ax4.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax4.set_ylim(-10*(len(sta_name2)),11)
ax4.set_yticks([])
ax4.grid(True)
ax4.annotate('D',xy = (-0.03,1),xycoords=ax4.transAxes,fontsize=14)
# ax4.tick_params(
#     axis='y',          # changes apply to the x-axis
#     which='both',      # both major and minor ticks are affected
#     bottom=False,      # ticks along the bottom edge are off
#     top=False,         # ticks along the top edge are off
#     labelbottom=False) # labels along the bottom edge are off
plt.figure(fig.number)
plt.savefig(Direc+'/CC_seismic_'+str(itime)+'.pdf')


# # plot spatial map    
# fig = plt.subplots(1,figsize=(15,15))
# img_extent = (-163,-150,52,59.5)
# ax=plt.axes(projection=ccrs.PlateCarree())
# ax.coastlines(resolution='50m', color='black', linewidth=1)
# plt.plot(obs.loc[:,3],obs.loc[:,2],'bo')
# plt.plot(trench[:,0],trench[:,1],'k-')
# for ista in range(len(station_name2)):
#     inum = np.where(station_name2[ista] == obs.loc[:,1])
#     plt.text(obs.loc[inum,3]-0.3,obs.loc[inum,2]+0.05,obs.loc[inum,1].to_string().split( )[1])
# for ista in range(len(cc_sta_name)):
#     inum = np.where(sta_name2[ista] == obs.loc[:,1])
#     plt.plot(obs.loc[inum,3],obs.loc[inum,2],'ro')
#     plt.text(obs.loc[inum,3]-0.3,obs.loc[inum,2]+0.05,obs.loc[inum,1].to_string().split( )[1],color = 'r')
# plt.title('itime:' +str(itime))
# # plt.savefig(Direc+'/CC_map_sta_'+str(itime)+'.pdf')
# plt.show()